In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Research/GAN/CycleGAN"

/content/drive/My Drive/Research/GAN/CycleGAN


In [3]:
%tensorflow_version 1.x
!pip install scipy==1.1.0
import scipy
print(scipy.__version__)
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
     |████████████████████████████████| 31.2MB 102kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
1.1.0
1.15.2


In [7]:
!bash ./download_datasets.sh facades

for details.

--2020-05-26 03:10:14--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/facades.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35135868 (34M) [application/zip]
Saving to: ‘./input/facades.zip’

./input/facades.zip 100%[===================>]  33.51M  37.3MB/s    in 0.9s    

2020-05-26 03:10:15 (37.3 MB/s) - ‘./input/facades.zip’ saved [35135868/35135868]

Archive:  ./input/facades.zip
   creating: ./input/facades/trainA/
  inflating: ./input/facades/trainA/110_A.jpg  
  inflating: ./input/facades/trainA/324_A.jpg  
  inflating: ./input/facades/trainA/353_A.jpg  
  inflating: ./input/facades/trainA/319_A.jpg  
  inflating: ./input/facades/trainA/20_A.jpg  
  inflating: ./input/facades/trainA/274_A.jpg  
  inflating: ./input/facades/trainA/107_A.jpg  
  inflating: ./inp

In [4]:
!ls -l input/facades/trainA | wc -l
!ls -l input/facades/testA | wc -l

401
107


In [0]:
DATASET_TO_SIZES = {
    'facades_train': 401,
    'facades_test': 107
}

DATASET_TO_IMAGETYPE = {
    'facades_train': '.jpg',
    'facades_test': '.jpg',
}

PATH_TO_CSV = {
    'facades_train': './input/facades/facades_train.csv',
    'facades_test': './input/facades/facades_test.csv',
}

In [0]:
import create_cyclegan_dataset
import importlib
importlib.reload(create_cyclegan_dataset)
from create_cyclegan_dataset import create_dataset


image_path_a = 'input/facades/trainA'
image_path_b = 'input/facades/trainB'
dataset_name = 'facades_train'
do_shuffle = 0
create_dataset(image_path_a, image_path_b, dataset_name, do_shuffle, DATASET_TO_SIZES, DATASET_TO_IMAGETYPE, PATH_TO_CSV)

In [0]:
from main import main

to_train = 1
log_dir = "output/cyclegan/facades"
config_filename = "configs/facades.json"
skip = True
checkpoint_dir= None#"output/cyclegan/facades/20200525-214032"
main(to_train, log_dir, config_filename, checkpoint_dir, skip, save_training_images=False)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.

100%|██████████| 401/401 [04:55<00:00,  1.36it/s]

Epoch 1/200


100%|██████████| 401/401 [01:46<00:00,  3.75it/s]


Epoch 2/200


...

100%|██████████| 401/401 [01:41<00:00,  3.94it/s]


Epoch 23/200


100%|██████████| 401/401 [01:41<00:00,  3.93it/s]


Epoch 24/200


 17%|█▋        | 69/401 [00:17<01:24,  3.93it/s]

In [0]:
from main import main

to_train = 0
log_dir = "output/cyclegan/facades"
config_filename = "configs/facades_test.json"
skip = True
checkpoint_dir= "output/cyclegan/facades/20200525-214032"
main(to_train, log_dir, config_filename, checkpoint_dir, skip)

/fs/junkfood/ssaxena1/miniconda3/envs/gan/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Testing the results
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
INFO:tensorflow:Restoring parameters from output/cyclegan/facades/20200525-214032/cyclegan-0


  0%|          | 0/107 [00:00<?, ?it/s]

Saving Images


100%|██████████| 107/107 [03:57<00:00,  2.21s/it]
